In [2]:
import re

# -------------------------------
# قائمة بالزوائد (المقدمات والنهايات)
# -------------------------------
prefixes = [
    "ال", "وال", "بال", "كال", "فال", "لل", "و", "ف", "ب", "ك", "ل", "س"
]

suffixes = [
    "ه", "ها", "ك", "ي", "كم", "نا", "هما", "هم", "كن", "ين", "ون", "ات", "ان", "تان", "تين", "ة", "ات"
]

# -------------------------------
# دالة تنظيف الكلمة من العلامات والزوائد
# -------------------------------
def normalize_arabic(word):
    word = re.sub(r'[ًٌٍَُِّْـ]', '', word)  # إزالة التشكيل
    word = re.sub(r'[^\u0600-\u06FF]', '', word)  # إزالة الرموز غير العربية
    return word

# -------------------------------
# دالة إزالة الزوائد
# -------------------------------
def remove_affixes(word):
    for pre in sorted(prefixes, key=len, reverse=True):
        if word.startswith(pre) and len(word) - len(pre) >= 3:
            word = word[len(pre):]
            break
    for suf in sorted(suffixes, key=len, reverse=True):
        if word.endswith(suf) and len(word) - len(suf) >= 3:
            word = word[:-len(suf)]
            break
    return word

# -------------------------------
# الأنماط الجذرية الشائعة (patterns)
# -------------------------------
patterns = [
    "مفعول", "فاعل", "فعول", "فعيلة", "افتعل", "استفعل", "تفاعل", "فعالة",
    "فعلى", "فعّل", "تفعّل", "انفعل", "افعل", "مفعل", "يفعل", "فعّل", "فعلة"
]

# -------------------------------
# استخراج الجذر عبر مقارنة الأنماط
# -------------------------------
def extract_root(word):
    for pattern in patterns:
        # تحويل النمط إلى شكل regex باستخدام ف ع ل كمثال
        regex = pattern.replace("ف", "(.?)").replace("ع", "(.?)").replace("ل", "(.?)")
        match = re.match(regex, word)
        if match:
            root = "".join(match.groups())
            root = re.sub(r'[^ءاأإآبتثجحخدذرزسشصضطظعغفقكلمنهوية]', '', root)
            if 3 <= len(root) <= 4:
                return root
    return word

# -------------------------------
# الدالة الرئيسية: Khoja Stemmer
# -------------------------------
def khoja_stem(word):
    word = normalize_arabic(word)
    word = remove_affixes(word)
    root = extract_root(word)
    return root

# -------------------------------
# اختبار عملي على فقرة عربية
# -------------------------------
text = """
المدارس يكتبون المعلمة مكتبة القراءة والمكتوبات فتعلم الطلاب الكتابة الصحيحة.
"""

words = [w for w in text.split() if len(w) > 2]

print("🔹 Khoja Stemmer Results:\n")
for w in words:
    print(f"{w:<15} → {khoja_stem(w)}")


🔹 Khoja Stemmer Results:

المدارس         → دار
يكتبون          → كتب
المعلمة         → علم
مكتبة           → كتب
القراءة         → قراء
والمكتوبات      → كتب
فتعلم           → تعلم
الطلاب          → طلاب
الكتابة         → كتاب
الصحيحة.        → صحيح
